In [1]:
# Import findspark and initialize.
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)

home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
home_sales_df.show(5)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [4]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView("home_sales")


In [5]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query = """
SELECT year(date) AS year, CAST(AVG(price) AS DECIMAL(10,2)) AS average_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY 1
ORDER BY year
"""
spark.sql(query).show()

+----+-------------+
|year|average_price|
+----+-------------+
|2019|    300263.70|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [8]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query = """
SELECT date_built, CAST(AVG(price) AS DECIMAL(10,2)) AS average_price
FROM home_sales
WHERE bedrooms = 3
AND bathrooms = 3
GROUP BY 1
ORDER BY average_price DESC
"""
spark.sql(query).show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2013|    295962.27|
|      2012|    293683.19|
|      2010|    292859.62|
|      2017|    292676.79|
|      2011|    291117.47|
|      2014|    290852.27|
|      2016|    290555.07|
|      2015|    288770.30|
+----------+-------------+



In [9]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
query = """
SELECT date_built, CAST(AVG(price) AS DECIMAL(10,2)) AS average_price
FROM home_sales
WHERE bedrooms = 3
AND bathrooms = 3
AND floors = 2
AND sqft_living >= 2000
GROUP BY 1
ORDER BY average_price DESC
"""
spark.sql(query).show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|    293965.10|
|      2010|    285010.22|
|      2017|    280317.58|
|      2011|    276553.81|
+----------+-------------+



In [10]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query = """
SELECT view, CAST(AVG(price) AS DECIMAL(10,2)) AS average_price
FROM home_sales
GROUP BY 1
HAVING average_price >= 350000
ORDER BY 1 DESC
"""
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|   1054325.60|
|  94|   1033536.20|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|   1072285.20|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|   1063498.00|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 1.0900142192840576 seconds ---


In [11]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [12]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [13]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

query = """
SELECT view, CAST(AVG(price) AS DECIMAL(10,2)) AS average_price
FROM home_sales
GROUP BY 1
HAVING average_price >= 350000
ORDER BY 1 DESC
"""
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|average_price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|   1054325.60|
|  94|   1033536.20|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|   1072285.20|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|   1063498.00|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 0.4932677745819092 seconds ---


In [14]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [15]:
# 11. Read the formatted parquet data.
home_sales_partitioned_df = spark.read.parquet("home_sales_partitioned")

In [16]:
# 12. Create a temporary table for the parquet data.
home_sales_partitioned_df.createOrReplaceTempView("home_sales_partitioned")

In [17]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

query = """
SELECT view, CAST(AVG(price) AS DECIMAL(10,2)) AS average_price
FROM home_sales_partitioned
GROUP BY 1
HAVING average_price >= 350000
ORDER BY 1 DESC
"""
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|   1054325.60|
|  94|   1033536.20|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|   1072285.20|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|   1063498.00|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 0.9588525295257568 seconds ---


In [18]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")

DataFrame[]

In [19]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False